In [8]:
from fishsense_api_sdk.client import Client
from pathlib import Path
from synology_api.filestation import FileStation
from fishsense_data_processing_workflow_worker.config import settings
from tqdm.notebook import tqdm
import pymupdf
import numpy as np
import cv2

In [2]:
URL = "http://localhost:8000"

NAS_HOST = "e4e-nas.ucsd.edu"
NAS_PORT = 6021

In [3]:

DATA_FOLDER = (Path("../data") / "fishsense" / "FishSense Lite Calibration Parameters").absolute()
OUTPUT_FOLDER = (Path("../output") / "pdfs").absolute()

DATA_FOLDER.mkdir(parents=True, exist_ok=True)
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

DATA_FOLDER.exists(), OUTPUT_FOLDER.exists()

(True, True)

In [4]:
filestation = FileStation(NAS_HOST, NAS_PORT, settings.e4e_nas.username, settings.e4e_nas.password, secure=True, cert_verify=False)

User logged in, new session started!


In [5]:
async with Client(URL) as client:
    dive_slates = await client.dive_slates.get()

len(dive_slates), dive_slates[0]

(11,
 DiveSlate(id=2, name='Tic-Tac-Toe 1', dpi=300, path='Dive Slate#1.pdf', created_at=datetime.datetime(2025, 9, 6, 3, 53, 10, 189717, tzinfo=TzInfo(0))))

In [10]:
for slate in tqdm(dive_slates):
    pdf_path = DATA_FOLDER / slate.path
    target_path = OUTPUT_FOLDER / f"{Path(slate.path).stem}.png"

    source_nas_path = f"/fishsense/Fishsense Lite Calibration Parameters/{slate.path}"
    print(f"Downloading {source_nas_path} to {pdf_path}")
    filestation.get_file(source_nas_path, "download", dest_path=str(pdf_path.parent))

    with pymupdf.open(pdf_path) as pdf_document:
        page: pymupdf.Page = pdf_document.load_page(0)
        pixmap: pymupdf.Pixmap = page.get_pixmap(dpi=slate.dpi)
        bytes = np.frombuffer(pixmap.samples, dtype=np.uint8)

        image = bytes.reshape(pixmap.height, pixmap.width, pixmap.n)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

        _, image = cv2.threshold(image, 125, 255, cv2.THRESH_BINARY)

        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    cv2.imwrite(str(target_path), image)
    print(f"Wrote {target_path}")

  0%|          | 0/11 [00:00<?, ?it/s]

Wrote /home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/scripts/../output/pdfs/Dive Slate#1.png
Wrote /home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/scripts/../output/pdfs/Dive Slate #2.png
Wrote /home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/scripts/../output/pdfs/Dive Slate #3.png
Wrote /home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/scripts/../output/pdfs/Dive Slate#4.png
Wrote /home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/scripts/../output/pdfs/Dive Slate #5.png
Wrote /home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/scripts/../output/pdfs/Dive Slate#6.png
Wrote /home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/scripts/../output/pdfs/SMILE vslate 1.png
Wrote /home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/scripts/../output/p